**Keras Tuner - Decides the number of hidden layers and the neuron in neural network**

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

C:\Users\Gaurav\AppData\Local\Temp\ipykernel_7216\1217278582.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
df = pd.read_csv("Real_Combine.csv")

In [14]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [15]:
X = df.iloc[:,:-1] # independent features
y = df.iloc[:,-1] # Dependent features

## Hyperparameters 
1. How many numbers of hidden layers we should have?
2. How many neurons we should have in hidden layers?
3. Learning rate

In [16]:
def build_model(hp):
    model = keras.Sequential()

    # Hidden layers
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(
            layers.Dense(
                units=hp.Int(
                    'units_' + str(i),
                    min_value=32,
                    max_value=512,
                    step=32
                ),
                activation='relu'
            )
        )

    # Output layer (ONLY ONCE)
    model.add(layers.Dense(1, activation='linear'))

    # Compile (ONLY ONCE)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-3, 1e-4])  # removed 1e-2 (too high)
        ),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error']
    )

    return model


In [17]:
tuner = RandomSearch(
    build_model,
    objective = 'val_mean_absolute_error',
    max_trials = 5,
    executions_per_trial = 3,
    directory = 'project_new',
    project_name= 'Air Quality Index'
) 

Reloading Tuner from project_new\Air Quality Index\tuner0.json


In [18]:
tuner.search_space_summary()

Search space summary
Default search space size: 20
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6 (Int

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

In [20]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))



Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
8                 |8                 |num_layers
448               |448               |units_0
416               |416               |units_1
0.0001            |0.0001            |learning_rate
128               |128               |units_2
96                |96                |units_3
160               |160               |units_4
160               |160               |units_5
32                |32                |units_6
128               |128               |units_7
352               |352               |units_8
128               |128               |units_9
96                |96                |units_10
224               |224               |units_11
352               |352               |units_12
128               |128               |units_13
512               |512               |units_14
448               |448               |units_15
64                |64                |units_16
512               |512        

C:\Users\Gaurav\Desktop\ML Projects\KrishNaik\env\Lib\site-packages\keras_tuner\src\engine\metrics_tracking.py:111: RuntimeWarning: All-NaN axis encountered
  np.nanmin(values) if self.direction == "min" else np.nanmax(values)


RuntimeError: Number of consecutive failures exceeded the limit of 3.


In [11]:
import numpy as np

print("X_train NaN:", np.isnan(X_train).sum())
print("X_test NaN:", np.isnan(X_test).sum())
print("y_train NaN:", np.isnan(y_train).sum())
print("y_test NaN:", np.isnan(y_test).sum())

print("X_train Inf:", np.isinf(X_train).sum())
print("X_test Inf:", np.isinf(X_test).sum())


X_train NaN: T      0
TM     0
Tm     0
SLP    0
H      0
VV     0
V      0
VM     0
dtype: int64
X_test NaN: T      0
TM     0
Tm     0
SLP    0
H      0
VV     0
V      0
VM     0
dtype: int64
y_train NaN: 0
y_test NaN: 1
X_train Inf: T      0
TM     0
Tm     0
SLP    0
H      0
VV     0
V      0
VM     0
dtype: int64
X_test Inf: T      0
TM     0
Tm     0
SLP    0
H      0
VV     0
V      0
VM     0
dtype: int64


In [12]:
import numpy as np

# Replace inf with NaN
df = df.replace([np.inf, -np.inf], np.nan)

# Drop all rows with any NaN
df = df.dropna()


In [10]:
tuner.results_summary()

Results summary
Results in project_new\Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 0 summary
Hyperparameters:
num_layers: 18
units_0: 416
units_1: 448
learning_rate: 0.0001
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
Score: nan

Trial 1 summary
Hyperparameters:
num_layers: 3
units_0: 288
units_1: 448
learning_rate: 0.0001
units_2: 160
units_3: 320
units_4: 128
units_5: 512
units_6: 32
units_7: 32
units_8: 288
units_9: 192
units_10: 224
units_11: 512
units_12: 64
units_13: 512
units_14: 320
units_15: 288
units_16: 32
units_17: 192
Score: nan

Trial 2 summary
Hyperparameters:
num_layers: 8
units_0: 448
units_1: 416
learning_rate: 0.0001
units_2: 128
units_3: 96
units_4: 160
units_5: 160
units_6: 32
units_7: 128
units_8: 352
units_9: 128
units_10: 96
units_11: 224
units_12: